In [ ]:
from fastapi import FastAPI, File, UploadFile
import cv2
import numpy as np
from detectron2.utils.visualizer import Visualizer
from detectron2.config import cfg
from detectron2 import model_zoo

# Replace with config file path (e.g., "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN.yaml")
cfg_path = "path/to/your/config.yaml"

# Load configuration and model
cfg.merge_from_file(cfg_path)
model = model_zoo.get(cfg.MODEL.WEIGHTS, train=False)

app = FastAPI()

def preprocess_image(image_bytes):
    # Read image from bytes
    image = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
    # Preprocess the image (resize) based on model requirements
    image = cv2.resize(image, (cfg.INPUT.IMAGE_SIZE, cfg.INPUT.IMAGE_SIZE))
    return image

def predict_objects(image):
    # Convert image to RGB format (BGR -> RGB for Detectron2)
    image = image[:, :, ::-1]
    # Prepare image as Detectron2 input format
    image = np.ascontiguousarray(image, dtype=np.float32)
    image_input = {"image": image}
    
    # Perform instance segmentation prediction
    predictions = model([image_input])[0]
    
    # Process detections (extract relevant information)
    boxes = []
    confidences = []
    class_ids = []
    masks = []
    for instance in predictions["instances"].pred_boxes:
        box = instance.get_bbox(flip=False).tolist()  # Adjust based on flip logic
        confidence = predictions["scores"][instance.pred_class].item()
        class_id = int(instance.pred_class)
        mask = predictions["instances"].pred_masks[0, instance.pred_class].cpu().numpy()
        boxes.append(box)
        confidences.append(confidence)
        class_ids.append(class_id)
        masks.append(mask)

    results = []
    for i in range(len(boxes)):
        x_min, y_min, x_max, y_max = boxes[i]
        class_name = model.cfg.MODEL.ROI_HEADS.NAME CLASSES[class_ids[i]]
        confidence = confidences[i]
        # Encode mask (consider RLE or PNG based on implementation)
        mask_encoded = cv2.imencode('.png', mask)[1].tobytes()  # Example PNG encoding
        results.append({"class": class_name, "score": confidence, "bbox": [x_min, y_min, x_max, y_max], "mask": mask_encoded})
    return results

@app.post("/upload")
async def upload_image(image: UploadFile = File(...)):
    # Read image content
    image_bytes = await image.read()
    # Preprocess image
    image = preprocess_image(image_bytes)
    # Perform instance segmentation
    results = predict_objects(image)
    return results

if __name__ == "__main__":
    import uvicorn
    uvicorn.run("main:app", host="0.0.0.0", port=8000)
